In [1]:
import telepot
from telepot.loop import MessageLoop
import pandas as pd
import pandas as pd
import numpy as np
import pickle
import threading
import glob
import re

stock_predictor_bot = telepot.Bot('359528427:AAHKD8rIiFpo0ufZTDvLRGQ9c6gTfFFfYq4')

In [ ]:
import logging
import os
import sys
# logger = logging.getLogger()
# logger.setLevel(logging.INFO)
# from pyspark.sql import Row
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")

# Add the py4j to the path.
# You may need to change the version number to match your install
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.6-src.zip'))

# Initialize PySpark to predefine the SparkContext variable 'sc'
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

In [ ]:
import pyspark

from pyspark.ml import PipelineModel
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressionModel
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

In [ ]:
def convert_float( x):
    try:
        x['Close'] = float(x['Close'])
        x['High'] = float(x['High'])
        x['Low'] = float(x['Low'])
        x['Open'] = float(x['Open'])
        return x
    except:
        x['Close'] = float(x['Close'])
        x['High'] = float(x['Close'])
        x['Low'] = float(x['Close'])
        x['Open'] = float(x['Close'])
        return x
    
stock_data = pd.DataFrame()
d_transformed = pd.DataFrame()
days = 30

def feature_ext(temp=None):
    temp['close_close_gap'] = (temp['Close'].shift(-1) - temp['Close']) / temp['Close']
    temp['close_open_gap'] = (temp['Open'].shift(-1) - temp['Close']) / temp['Close']
    temp['close_high_gap'] = (temp['High'].shift(-1) - temp['Close']) / temp['Close']
    temp['close_low_gap'] = (temp['Low'].shift(-1) - temp['Close']) / temp['Close']
    temp.drop(labels=["High","Low","Open","Close", "Volume"], axis=1,inplace=True)
    return temp

list_stock = glob.glob('until31_2017/*.csv')
all_stock = pd.DataFrame(columns=['stock_code','Date','close_open_gap','close_high_gap','close_low_gap','close_close_gap','Volume'])

def get_feature(code, date, days_to_be_stacked=2) :
    try :
        df = pd.read_csv('until31_2017/' + code + '.csv')
        selected_index = df.index[df['Date'] == '8-Dec-17'].tolist()
        
        if len(selected_index) == 0 :
            return [];
        
        selected_index = selected_index[0]
        df = df[selected_index + 1 : selected_index + days_to_be_stacked + 2]
        df = df.reset_index().drop('index',axis=1)
        df = df.drop(labels=['Date'], axis=1)
        df = df.apply(lambda x:convert_float(x),axis=1)
        feature_ext(df)
        df = df[0:days_to_be_stacked]
        df = df.stack().to_frame().T
        df.columns = ['{}_{}'.format(*c) for c in df.columns]
#         return df
        return df.values[0];
    
    except Exception as e:
        print("exception when get feature " + str(e))
        return []

In [ ]:
def handle_message(msg):
    content_type, chat_type, chat_id = telepot.glance(msg)

    message = ""
    username = msg["from"]["first_name"]
    if content_type == 'text':
        message = msg["text"]
        
        if 'predict' in message :
            stock_predictor_bot.sendMessage(chat_id, 'prediction starting.. ')
            predict_stock(chat_id, message)

def predict_stock(chat_id, msg):
    try :
        temp = msg.split(" ")
        stock_code = temp[1]
        date = temp[2]
        stock_predictor_bot.sendMessage(chat_id, 'gathering features for stock ' + stock_code)
        feature = get_feature(stock_code, date)
        
        if len(feature) == 0 :
            stock_predictor_bot.sendMessage(chat_id, 'invalid data')
        
        df = sc.parallelize([{'features':feature.tolist()}]).toDF()
        list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
        final_data = df.select(list_to_vector_udf(df["features"]).alias("features"))
        savedModel = PipelineModel.load('gbtModel_3_features')
        
        stock_predictor_bot.sendMessage(chat_id, 'predicting for stock ' + stock_code)
        predictions = savedModel.transform(final_data)
        result = round(predictions.collect()[0]['prediction'], 4)
        stock_predictor_bot.sendMessage(chat_id, 'prediction result ' + str(result))
    except Exception as e :
        print("exception when predict stock " + str(e))
    
def start():
    MessageLoop(stock_predictor_bot, handle_message).run_as_thread()
    print ('Stock Predictor Bot Listening ...')

In [ ]:
start()

import time
while 1:
    time.sleep(100)

Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/telepot/loop.py", line 148, in run_forever
    self._bot.scheduler.run_as_thread()
  File "/usr/local/lib/python3.5/dist-packages/telepot/__init__.py", line 404, in run_as_thread
    self.daemon = True
  File "/usr/lib/python3.5/threading.py", line 1139, in daemon
    raise RuntimeError("cannot set daemon status of active thread")
RuntimeError: cannot set daemon status of active thread



Food Bot Listening ...


/home/tryan/spark_tensor/spark-2.3.0-bin-hadoop2.7/python/pyspark/sql/session.py:360: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "
